
YT Video Start: https://youtu.be/N1FM-PcVXNA?si=LTwvfIozUsS6clqF&t=185


db_id = "819025bd-668f-4546-99b7-b55b01bc1667"



In [4]:
!pip install langchain langgraph cassio

In [ ]:
import cassio
import json

# Retreive the tokens from file
with open("tokens.json", "r") as f:
  tokens = json.load(f)

ASTRADB_TOKEN = tokens["ASTRADB_TOKEN"]
TEST_DB_ID = tokens["TEST_DB_ID"]

# Initialize with the connection
cassio.init(token=ASTRADB_TOKEN, database_id=TEST_DB_ID)


AstraCS:HmxfrEAoahwqtziKZBzdQZzO:f6e93845f199e6fa7d1167b008676166da2bc7d34a6b140f3c1994456e159781 819025bd-668f-4546-99b7-b55b01bc1667
